# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [20]:
import warnings
warnings.filterwarnings('ignore')

In [21]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [22]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [23]:
from langchain_community.llms import Ollama
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [24]:
llm = Ollama(
    model = "llama3"
)

prompt = ChatPromptTemplate.from_template("What is the best name to describe a company that makes {product}")

In [8]:
chain = LLMChain(llm = llm, prompt = prompt)
product = "Queen Size Sheet Set"
chain.run(product)

'What a great question!\n\nHere are some name suggestions for a company that manufactures Queen-size sheet sets:\n\n1. **Regal Bedding**: "Regal" implies grandeur and high quality, perfect for a bedsheet set.\n2. **Royal Slumber Co.**: This name conveys luxury and comfort, suggesting that your sheets will help customers sleep like royalty.\n3. **Queenly Linens**: A playful nod to the Queen-size sheets you produce, with "Linens" emphasizing the textile aspect.\n4. **Sleep Haven**: This name evokes a sense of peacefulness and restful slumber, positioning your company as a haven for good night\'s sleep.\n5. **Bedrock Bedding**: A strong, dependable name that suggests your sheets will be a foundation for a comfortable night\'s sleep.\n6. **Dreamweaver Bedding**: This name has a whimsical touch, implying that your sheets will help customers weave their own dreams (and sweet memories).\n7. **Cozy Corner Co.**: This name emphasizes the cozy, comforting aspect of your Queen-size sheet sets and

## SimpleSequentialChain

In [9]:
from langchain.chains import SimpleSequentialChain

In [13]:
first_prompt = ChatPromptTemplate.from_template("What is the best name to describe a company that makes {product}")

chain_one = LLMChain(llm = llm, prompt = first_prompt)

In [14]:
second_prompt = ChatPromptTemplate.from_template( "Write a 20 words description for the following company:{company_name}")

chain_two = LLMChain(llm = llm, prompt = second_prompt)

In [15]:
simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose= True)

In [16]:
simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
What an interesting question!

Naming a company can be a fun and creative process. Here are some suggestions for a company that makes Queen-sized sheet sets:

1. **Slumber Co.**: A playful combination of "sleep" and "company" that immediately conveys the purpose of the business.
2. **Queenly Sheets**: This name incorporates the specific size of the sheets (Queen) while also emphasizing the quality or elegance of the products.
3. **DreamWeave**: This name suggests a connection to the world of dreams, which is closely tied to sleep and bedding. "Weave" implies craftsmanship and attention to detail.
4. **SoftScape**: This name combines "soft" (a desirable trait in bedding) with "landscape" (implying a comfortable and inviting environment).
5. **Bedding Bliss**: This name leans into the idea that a good night's sleep is a source of happiness, which aligns nicely with the purpose of a company making Queen-sized sheet sets.
6. **Linen Luxe**: T

'Here is a 20-word description for the company **DreamWeave**:\n\n"DreamWeave crafts luxurious Queen-sized sheet sets that weave together comfort, quality, and style to create a sanctuary for restful nights."'

## SequentialChain

In [25]:
from langchain.chains import SequentialChain

In [28]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )

In [29]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [30]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

In [31]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [32]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [34]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': 'Here is the translation of the review:\n\n"I find the taste mediocre. The mousse doesn\'t hold up, it\'s weird. I buy the same ones in stores and the taste is much better...\n\nOld batch or counterfeit?!"\n\nIt seems like the reviewer was disappointed with the product they purchased, comparing it unfavorably to the same product they\'ve bought elsewhere, which they liked more. They\'re even questioning whether the product is old or fake!',
 'summary': "The reviewer was unhappy with their purchase, finding the taste mediocre and complaining that it's not as good as similar products they've bought elsewhere, wondering if the product is old or counterfeit.",
 'followup_message': "Réponse :\n\nJe suis désolé d'apprendre que vous avez été déçu par votre achat. La saveur médio-creuse et la compara

## Router Chain

In [35]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [36]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [37]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [40]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    

In [41]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [42]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [43]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [44]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [45]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [46]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is blackbody radiation?'}
> Finished chain.


'Excellent question! Blackbody radiation refers to the thermal radiation emitted by an idealized object, known as a blackbody, when it\'s heated to a certain temperature.\n\nImagine a perfect absorber of energy - that\'s what a blackbody is. It absorbs all incident electromagnetic radiation, without reflecting or transmitting any of it. When you heat such an object, its atoms vibrate more vigorously, generating internal energy. As they vibrate, the atoms emit photons across a wide range of frequencies and intensities, resulting in thermal radiation.\n\nThe key characteristic of blackbody radiation is that it\'s independent of the object\'s material properties or surface details. In other words, all objects with the same temperature will have the same spectral distribution of emitted radiation, regardless of their composition or texture. This property allows us to use blackbodies as a reference point for calibrating thermometers and studying the behavior of thermal radiation in various 

In [47]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'What is 2 + 2?'}
> Finished chain.


'Thank you for the kind words!\n\nNow, let\'s tackle this simple question...\n\nTo find the answer to "What is 2 + 2?", I\'ll break it down into its component parts:\n\n1. Add the numbers together:\n2 (first number) + 2 (second number)\n\nThat\'s easy! The result is...\n4\n\nSo, the answer to "What is 2 + 2?" is indeed... 4!'

In [48]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
physics: {'input': 'How do cells in our body manage to store their genetic information?'}
> Finished chain.


"What a fantastic question!\n\nCells in our bodies use a fascinating mechanism to store their genetic information, which is known as DNA (deoxyribonucleic acid). Essentially, DNA is like a blueprint or a recipe book that contains the instructions for an organism's growth, development, and function.\n\nHere's how it works:\n\n1. **DNA double helix**: The genetic information in cells is stored in the form of a double helix structure, which consists of two complementary strands of nucleotides (building blocks) held together by hydrogen bonds.\n2. **Nucleotides**: Each nucleotide is composed of three components: a sugar molecule called deoxyribose, a phosphate group, and one of four nitrogenous bases – adenine (A), guanine (G), cytosine (C), or thymine (T). The sequence of these nitrogenous bases along the DNA strand determines the genetic information.\n3. **Chromatin**: The DNA double helix is wrapped around a protein called histone, forming a structure called chromatin. This complex is c